In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm, ParameterSweep

import numpy as np

np.set_printoptions(threshold=8, edgeitems=2)

# Define HilbertSpace object: two transmons coupled to an oscillator


tmon1 = scq.TunableTransmon(
    EJmax=40.0,
    EC=0.2,
    d=0.1,
    flux=0.23,
    ng=0.3,
    ncut=40,
    truncated_dim=3,     # after diagonalization, we will keep 3 levels
    id_str="tmon1"       # optional, used for referencing from within
                         # ParameterSweep or HilbertSpace
)

tmon2 = scq.TunableTransmon(
    EJmax=15.0,
    EC=0.15,
    d=0.2,
    flux=0.0,
    ng=0.0,
    ncut=30,
    truncated_dim=3,
    id_str="tmon2"
)

resonator = scq.Oscillator(
    E_osc=4.5,
    truncated_dim=4    # up to 3 photons (0,1,2,3)
)

hilbertspace = scq.HilbertSpace([tmon1, tmon2, resonator])


g1 = 0.1  # coupling resonator-CPB1 (without charge matrix elements)
g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)

hilbertspace.add_interaction(
    g_strength = g1,
    op1 = tmon1.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True,
    id_str="tmon1-resonator"  # optional keyword argument
)

hilbertspace.add_interaction(
    g_strength = g2,
    op1 = tmon2.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True,
    id_str="tmon2-resonator"  # optional keyword argument
)

# Set up parameter name and values
pname1 = 'flux'  
flux_vals = np.linspace(0.0, 2.0, 171)
pname2 = 'ng'
ng_vals = np.linspace(-0.5, 0.5, 49)

# combine into a dictionary
paramvals_by_name = {pname1: flux_vals, pname2: ng_vals}


area_ratio = 1.2

def update_hilbertspace(flux, ng):  # function that defines how Hilbert space components are updated
    tmon1.flux = flux
    tmon2.flux = area_ratio * flux
    tmon2.ng = ng

# dictionary with information on which subsystems are affected by changing parameters    
subsys_update_info = {pname1: [tmon1, tmon2],
                      pname2: [tmon2]}

    
# create the ParameterSweep    
sweep = ParameterSweep(
    hilbertspace=hilbertspace,
    paramvals_by_name=paramvals_by_name,
    update_hilbertspace=update_hilbertspace,
    evals_count=20,
    subsys_update_info=subsys_update_info,
    num_cpus=4
)

Parallel compute bare eigensys for subsystem tmon1 [num_cpus=4]

Parallel compute bare eigensys for subsystem tmon2 [num_cpus=4]

Parallel compute bare eigensys for subsystem Oscillator_1 [num_cpus=4]

Parallel compute dressed eigensys [num_cpus=4]

## ParameterSweep data

Much of the computed data that is stored and immediately retrievable after this sweep by accessing the `ParameterSweep` object like a dictionary with the following string keys:


+-------------------------------+------------------------------------------------------------------------+
| string keyword                |    data                                                                |
+===============================+========================================================================+
| `"evals"`, `"evecs"`          | dressed eigenenergies and eigenstates as `NamedSlotsNdarray`           |
|                               | eigenstates are decomposed in the bare product-state basis of the      |
|                               | non-interacting subsystems' eigenbases                                 |
+-------------------------------+------------------------------------------------------------------------+
| `"bare_evals"`, `"bare_evecs"`| eigenenergies and eigenstates for each subsystem as `NamedSlotsNdarray`|         
|                               | The arrays first axis (name: `"subsys"` enumerates the subsystems      |
+-------------------------------+------------------------------------------------------------------------+
| `"lamb"`, `"chi"`, `"kerr"`   | dispersive energy coefficients as `NamedSlotsNdarray`                  |
|                               | (see description below)                                                |
+-------------------------------+------------------------------------------------------------------------+


Data are returned as a `NamedSlotsNdarray` which behaves like a regular numpy array:

In [4]:
sweep["evals"]

NamedSlotsNdarray([[[-48.9786042 , -45.0278778 , ..., -29.62856751,
                     -29.01322673],
                    [-48.97847617, -45.02774984, ..., -29.62844062,
                     -29.0131366 ],
                    ...,
                    [-48.98107747, -45.03034988, ..., -29.63101896,
                     -29.0149706 ],
                    [-48.98131859, -45.03059088, ..., -29.63125795,
                     -29.01514089]],

                   [[-48.93975166, -44.99058959, ..., -29.59671601,
                     -28.98017335],
                    [-48.93962364, -44.99046163, ..., -29.59658911,
                     -28.98008301],
                    ...,
                    [-48.94222494, -44.99306166, ..., -29.59916745,
                     -28.98192134],
                    [-48.94246606, -44.99330267, ..., -29.59940645,
                     -28.98209203]],

                   ...,

                   [[-40.70088901, -38.21167383, ..., -23.61188977,
                     